# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846853615463                   -0.70    4.8         
  2   -7.852317546325       -2.26       -1.53    1.0   24.7ms
  3   -7.852617054365       -3.52       -2.56    1.5   27.8ms
  4   -7.852646071871       -4.54       -2.91    2.8   96.5ms
  5   -7.852646539140       -6.33       -3.23    1.0   25.3ms
  6   -7.852646680229       -6.85       -4.14    1.0   26.3ms
  7   -7.852646686612       -8.19       -5.33    2.2   32.8ms
  8   -7.852646686726       -9.94       -5.57    2.2   32.7ms
  9   -7.852646686728      -11.84       -5.66    1.5   28.5ms
 10   -7.852646686730      -11.68       -6.35    1.0   25.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846821281883                   -0.70           4.5         
  2   -7.852551978833       -2.24       -1.63   0.80    2.2    266ms
  3   -7.852639172175       -4.06       -2.74   0.80    1.0   23.6ms
  4   -7.852646430734       -5.14       -3.36   0.80    1.8   28.7ms
  5   -7.852646677805       -6.61       -4.26   0.80    2.0   29.8ms
  6   -7.852646686590       -8.06       -4.77   0.80    1.8   27.6ms
  7   -7.852646686720       -9.89       -5.68   0.80    1.2   84.4ms
  8   -7.852646686730      -11.01       -6.49   0.80    2.5   30.4ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.370577e+01     3.410243e+00
 * time: 0.061383962631225586
     1     1.076314e+00     1.748183e+00
 * time: 0.36644792556762695
     2    -1.878391e+00     2.025982e+00
 * time: 0.39095091819763184
     3    -4.042782e+00     1.536993e+00
 * time: 0.4257519245147705
     4    -5.365279e+00     1.357966e+00
 * time: 0.46056199073791504
     5    -7.006754e+00     7.706095e-01
 * time: 0.4950449466705322
     6    -7.572176e+00     3.477310e-01
 * time: 0.5297720432281494
     7    -7.735940e+00     1.502723e-01
 * time: 0.5537838935852051
     8    -7.799123e+00     1.166710e-01
 * time: 0.5778200626373291
     9    -7.828718e+00     7.109661e-02
 * time: 0.6017711162567139
    10    -7.841271e+00     6.723168e-02
 * time: 0.6264369487762451
    11    -7.845479e+00     5.281207e-02
 * time: 0.6504299640655518
    12    -7.847657e+00     3.171185e-02
 * time: 0.6752779483795166
    13    -7.849599e+00     2.166226e-02
 * time: 0.69949

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846869406279                   -0.70    4.5         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645915911                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.95s
  3   -7.852646686730      -13.30       -7.25    175ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 5.168585818704698e-7
|ρ_newton - ρ_scfv| = 9.13546863533761e-8
|ρ_newton - ρ_dm|   = 2.5771067077956973e-9
